#### Enviroment
Python 3.11.0

-----------------------------

### Libraries

In [1]:
from tabula.io import read_pdf
import pandas as pd
import tabula
import os

In [2]:
print(pd.options.display.max_rows) 

60


#### This chunk of the code is for you to know how to read PDF files but they are not going to be availabe becase contains sensible informatios of my number accounts; hoever you can use it to create the csv file to work with.


In [5]:
dir = os.getcwd()
dir_path = dir+'\Extractos'   # Just to find the path where de bank resume of expenses are available

lista_pdf = []  # List to store de name of .pdf

for files in os.listdir(dir_path):
    if files.endswith('pdf'):  #just files *.pdf
        lista_pdf.append(files)
    
print(lista_pdf)

['Banco en Línea - Santander1.pdf', 'Banco en Línea - Santander2.pdf', 'Banco en Línea - Santander3.pdf', 'Banco en Línea - Santander4.pdf']


### Transform data to CSV using the librarie Tabula 

In [6]:
dir = os.getcwd()
dir_path = dir+'\Extractos\\'  

for df in lista_pdf:
   tabula.convert_into(dir_path + df, "output"+df+".csv", output_format="csv", pages='all')

#### Conver all files to CSV format

In [7]:
dir_path = os.getcwd()

lista_csv = []

for files in os.listdir(dir_path):
    if files.endswith('csv'):
        lista_csv.append(files)
    
print(lista_csv)

['outputBanco en Línea - Santander1.pdf.csv', 'outputBanco en Línea - Santander2.pdf.csv', 'outputBanco en Línea - Santander3.pdf.csv', 'outputBanco en Línea - Santander4.pdf.csv']


### Cleancy 
in this part we are going to prepare de files to be correctly read
it involves to erase some rows because the pdf files have certain lines of information at the begining and end that is not necesary and might make the data dirty

In [9]:



colnames = ['Fecha', 'Salida', 'Entrada',  'Descripción','Documento']

df_s0 =  pd.read_csv(lista_csv[0], encoding='Latin1',skiprows=[0,1], skipfooter=4, engine='python',names=colnames)
df_s1 =  pd.read_csv(lista_csv[1], encoding='Latin1',skiprows=[0,1], skipfooter=4,engine='python',  names=colnames)
df_s2 =  pd.read_csv(lista_csv[2], encoding='Latin1',skiprows=[0,1],skipfooter=4, engine='python', names=colnames)
df_s3 =  pd.read_csv(lista_csv[3], encoding='Latin1',skiprows=[0,1], skipfooter=4, engine='python', names=colnames)

60


#### Next process is to cancatenate all the files so we can work just with one , operartions are easy that way in this case

In [12]:
df_s_all = pd.concat([df_s0,df_s1,df_s2,df_s3])
df_s_all.head()

,Fecha,Salida,Entrada,Descripción,Documento
0,03/01/2022,763.400,NaN,Transf. Internet a otro Bancos,0
1,03/01/2022,189.390,NaN,Traspaso Internet a T. Crédito,0
2,03/01/2022,30.092,NaN,Compra TOTTUS VICUNA MAC,0
3,03/01/2022,12.210,NaN,Compra BEST FOR PETS,0
4,03/01/2022,9.360,NaN,Compra SUPERMERCADO LOS,0


#### From this point on we are going to work just with purchase because de propose of this analisys has to be with how much I am spending and in what

In [33]:
df_purchase = df_s_all[df_s_all['Descripción'].str.contains('Compra') ]

In [34]:
df_purchase.to_csv('Purchase.csv')


#### df_bnk_purchases is the dataset to work, it doesn't contain sensible data so it will be available in this repository of Github

In [20]:
df_bnk_purchases= pd.read_csv("Purchase.csv")

In [21]:
# Lets check the amount of NA in each of the columns of the dataset
df_bnk_purchases.isnull().sum()

Unnamed: 0       0
Fecha            0
Salida           0
Entrada        487
Descripción      0
Documento        0
dtype: int64

In [22]:
df_bnk_purchases.head()

,Unnamed: 0,Fecha,Salida,Entrada,Descripción,Documento
0,2,03/01/2022,30.092,NaN,Compra TOTTUS VICUNA MAC,0
1,3,03/01/2022,12.210,NaN,Compra BEST FOR PETS,0
2,4,03/01/2022,9.360,NaN,Compra SUPERMERCADO LOS,0
3,5,03/01/2022,8.350,NaN,Compra STARBUCKS ESTADO,0
4,6,03/01/2022,3.000,NaN,Compra POLLA CHILENA,0


##### As we can see there is NA data in the column Entrada this is normal because the movments we are analysing are outcomes, in fact we can drop this column later since it is not necesary

#### On the other hand the column Descripcion has importan information that can help uo to segment information later for example:
- Transferences
- Who I transfered money
- Purchases
- Places where I purchase

In [23]:
# Lets proced spliting the column
purchases_des = df_bnk_purchases["Descripción"].str.split(' ', 2  ,expand=True).reset_index(drop=True)
purchases_des.head(10)

C:\Users\yohan\AppData\Local\Temp\ipykernel_7656\1086343862.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  purchases_des = df_bnk_purchases["Descripción"].str.split(' ', 2  ,expand=True).reset_index(drop=True)


,0,1,2
0,Compra,TOTTUS,VICUNA MAC
1,Compra,BEST,FOR PETS
2,Compra,SUPERMERCADO,LOS
3,Compra,STARBUCKS,ESTADO
4,Compra,POLLA,CHILENA
5,Compra,NOTARIA,ELENA TOR
6,Compra,PEPE,None
7,Compra,NOTARIA,LASCAR
8,Compra,SN,CAMILO METRO I
9,Compra,SODIMAC,PARQUE AR


#### Most of the information in the Descripción column has three states 0= type, 1 =Category and 3 = place
Lets replace the names of the columns

In [24]:
purchases_ren = purchases_des.rename(columns= {0:'Tipo',1:'Categoria',2:'Lugar'})
purchases_ren.head(10)

,Tipo,Categoria,Lugar
0,Compra,TOTTUS,VICUNA MAC
1,Compra,BEST,FOR PETS
2,Compra,SUPERMERCADO,LOS
3,Compra,STARBUCKS,ESTADO
4,Compra,POLLA,CHILENA
5,Compra,NOTARIA,ELENA TOR
6,Compra,PEPE,None
7,Compra,NOTARIA,LASCAR
8,Compra,SN,CAMILO METRO I
9,Compra,SODIMAC,PARQUE AR


In [25]:
# Now lets join the descriprions columns to the dataset
df_purchases_join = df_bnk_purchases.join(purchases_ren).reset_index(drop=True)
df_purchases_join.head()

,Unnamed: 0,Fecha,Salida,Entrada,Descripción,Documento,Tipo,Categoria,Lugar
0,2,03/01/2022,30.092,NaN,Compra TOTTUS VICUNA MAC,0,Compra,TOTTUS,VICUNA MAC
1,3,03/01/2022,12.210,NaN,Compra BEST FOR PETS,0,Compra,BEST,FOR PETS
2,4,03/01/2022,9.360,NaN,Compra SUPERMERCADO LOS,0,Compra,SUPERMERCADO,LOS
3,5,03/01/2022,8.350,NaN,Compra STARBUCKS ESTADO,0,Compra,STARBUCKS,ESTADO
4,6,03/01/2022,3.000,NaN,Compra POLLA CHILENA,0,Compra,POLLA,CHILENA


In [26]:
# Let see the shape of the Dataset
df_purchases_join.shape

(487, 9)

#### Another split that can be usefull is Date, we have to separate year, month and day in order to create a granular analisys and see if there is a patern in the way I spend money

In [27]:
# To see if I can split dates I have to convert Fecha into a date type, for now is an object
df_purchases_join.dtypes

Unnamed: 0       int64
Fecha           object
Salida         float64
Entrada        float64
Descripción     object
Documento        int64
Tipo            object
Categoria       object
Lugar           object
dtype: object

In [28]:
df_purchases_join['Fecha'] = pd.to_datetime(df_purchases_join['Fecha'] , dayfirst=True)

In [29]:
df_purchases_join.dtypes

Unnamed: 0              int64
Fecha          datetime64[ns]
Salida                float64
Entrada               float64
Descripción            object
Documento               int64
Tipo                   object
Categoria              object
Lugar                  object
dtype: object

#### There are a couple of columns that we are not going to use because they don't have relevant information

In [30]:
df_purchases_join.drop(['Descripción','Entrada','Documento'],  axis='columns', inplace=True)
df_purchases_join.head()

,Unnamed: 0,Fecha,Salida,Tipo,Categoria,Lugar
0,2,2022-01-03,30.092,Compra,TOTTUS,VICUNA MAC
1,3,2022-01-03,12.210,Compra,BEST,FOR PETS
2,4,2022-01-03,9.360,Compra,SUPERMERCADO,LOS
3,5,2022-01-03,8.350,Compra,STARBUCKS,ESTADO
4,6,2022-01-03,3.000,Compra,POLLA,CHILENA


#### Lets proceed with the split of date

In [33]:
df_purchases_join['year']= df_purchases_join['Fecha'].dt.year
df_purchases_join['month']= df_purchases_join['Fecha'].dt.month
df_purchases_join['semana']= df_purchases_join['Fecha'].dt.week
df_purchases_join['day']= df_purchases_join['Fecha'].dt.day


C:\Users\yohan\AppData\Local\Temp\ipykernel_7656\3569623546.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_purchases_join['semana']= df_purchases_join['Fecha'].dt.week


In [40]:
df_purchases_join.head(5)

,Unnamed: 0,Fecha,Salida,Tipo,Categoria,Lugar,year,month,semana,day
0,2,2022-01-03,30.092,Compra,TOTTUS,VICUNA MAC,2022,1,1,3
1,3,2022-01-03,12.210,Compra,BEST,FOR PETS,2022,1,1,3
2,4,2022-01-03,9.360,Compra,SUPERMERCADO,LOS,2022,1,1,3
3,5,2022-01-03,8.350,Compra,STARBUCKS,ESTADO,2022,1,1,3
4,6,2022-01-03,3.000,Compra,POLLA,CHILENA,2022,1,1,3


#### Lets create some data visualization just to give us an idea of what we are dealing with

In [44]:
# Skimpy is a librarie that create a framework for us to help with general observation 
#pip install skimpy

In [45]:
from skimpy import skim

skim(df_purchases_join)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 487    │ │ int32       │ 5     │                                                          │
│ │ Number of columns │ 10     │ │ string      │ 3     │                                                          │
│ └───────────────────┴────────┘ │ datetime64  │ 1     │                                                          │
│                                │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA   ┃ NA %    ┃ mean    ┃ sd      ┃ p0      ┃ p25     ┃ p75     ┃ p100    ┃ hist     ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩  │
│ │ Unnamed: 0         │    0 │       0 │     100 │      56 │       0 │      52 │     150 │     200 │  ▇▆▇▇█▆  │  │
│ │ Salida             │    0 │       0 │      51 │     140 │       1 │     4.4 │      29 │     900 │    █     │  │
│ │ year               │    0 │       0 │    2000 │    0.26 │    2000 │    2000 │    2000 │    2000 │  █    ▁  │  │
│ │ month              │    0 │       0 │     5.6 │     3.7 │       1 │       2 │       9 │      12 │  █▄▅▃▅▃  │  │
│ │ semana             │    0 │       0 │      23 │      16 │       1 │       7 │      38 │      52 │  █▃▅▂▅▃  │  │
│ │ day                │    0 │       0 │      16 │     8.6 │       1 │       9 │      24 │      31 │  ▅▅▄█▇▆  │  │
│ └────────────────────┴──────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴──────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name            ┃ NA     ┃ NA %      ┃ first               ┃ last                ┃ frequency        ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ Fecha                  │      0 │         0 │     2022-01-03      │     2023-01-30      │ None             │  │
│ └────────────────────────┴────────┴───────────┴─────────────────────┴─────────────────────┴──────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Tipo                      │       0 │          0 │                            1 │                      490 │  │
│ │ Categoria                 │       0 │          0 │                            1 │                      490 │  │
│ │ Lugar                     │      29 │          6 │                            1 │                      490 │  │
│ └───────────────────────────┴─────────┴────────────┴──

### What we are able to deduce is:
- In Salda the mean is 51 and the sd us high = 140
- The four quartiles of column Salida have a big gap between them so it is necesary to create segmentations to cluster groups
- We hace sone NA in column lugar maybe because in the split done before it create spaces in blank